In [70]:
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score as accuracy
files = list(Path("../cls-eval/NLI/").rglob("*.csv"))
files = [str(x) for x in files]
files

['../cls-eval/NLI/crowd/revised_hypothesis/2_test_llm_eval_classifier (2).csv',
 '../cls-eval/NLI/crowd/revised_premise/1_test_llm_eval_classifier (1).csv']

In [71]:
pd.read_csv(files[-1])

,Unnamed: 0,original_sentence1,original_sentence2,original label,contrast label,contrast text,label
0,0,A man with a beard is talking on the cellphone...,A man is prone on the street while another man...,entailment,contradiction,A man with a beard is talking on the cellphone...,2.0
1,1,A man with a beard is talking on the cellphone...,A man is prone on the street while another man...,entailment,neutral,A man with a beard is talking on the cellphone...,2.0
2,2,A kid slides down a yellow slide into a swimmi...,The child slides into the pool.,entailment,contradiction,A kid slides down a yellow slide onto the ground.,2.0
3,3,A kid slides down a yellow slide into a swimmi...,The child slides into the pool.,entailment,neutral,A kid slides down a yellow slide.,2.0
4,4,A woman talks on a cellphone while sitting in ...,She has a conversation on her phone outdoors,entailment,contradiction,A woman talks on a cellphone while sitting in ...,1.0
...,...,...,...,...,...,...,...
795,795,"A man wearing black pants, an orange and brown...",The bandana is expensive.,contradiction,entailment,"A man wearing black pants, an orange and brown...",1.0
796,796,At least eight people are sitting at a table i...,People sitting on the floor,contradiction,neutral,At least eight tall people are sitting at a ta...,2.0
797,797,At least eight people are sitting at a table i...,People sitting on the floor,contradiction,entailment,At least eight people are sitting at a table i...,2.0
798,798,A photographer and assistant take a photo of a...,The photographer is taking a photo of a dog.,contradiction,entailment,A photographer and assistant take a photo of t...,2.0


In [72]:
def map_label(x):
    # 1 - Entailment, 2 - Contradiction, 3 - Neutral
    #print(x)
    if x == 1:
        return 'entailment'
    elif x == 2:
        return 'contradiction'
    elif x == 3:
        return 'netural'
    else:
        raise ValueError


def get_result(files): 

    dict = {
        'LLM' : [], 
        'part': [], 
        'acc' : [], 
    }


    for f in files:
        llm_name = f.split('/')[-3]
        part =  f.split('/')[-2]

    
        dict['LLM'].append(llm_name)
        dict['part'].append(part)


        df = pd.read_csv(f)
        df = df.dropna()
        df['decoded label'] = df.apply(lambda x: map_label(x['label']), axis=1)

        dict['acc'].append(accuracy(df['contrast label'], df['decoded label'])*100)

    df_results = pd.DataFrame.from_dict(dict).round(2)
    return df_results.sort_values(by=['part', 'LLM'])

In [73]:
mean_results = get_result(files)
mean_results.columns

Index(['LLM', 'part', 'acc'], dtype='object')

In [74]:
mean_results

,LLM,part,acc
0,crowd,revised_hypothesis,58.32
1,crowd,revised_premise,50.88


In [75]:
table = mean_results[['LLM', 'part', 'acc']]
print(table.to_latex(index=False))

\begin{tabular}{llr}
\toprule
  LLM &               part &   acc \\
\midrule
crowd & revised\_hypothesis & 58.32 \\
crowd &    revised\_premise & 50.88 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2722802/466383237.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(table.to_latex(index=False))
